In [11]:
import os

# Set the environment variables for XRT
xrt_213_path = "/home/csl/yeop/XRT/build/Debug/opt/xilinx/xrt"

os.environ["XILINX_XRT"] = xrt_213_path
os.environ["LD_LIBRARY_PATH"] = f"{xrt_213_path}/lib:" + os.getenv("LD_LIBRARY_PATH", "")
os.environ["PATH"] = f"{xrt_213_path}/bin:" + os.getenv("PATH", "")

In [12]:
!xbutil --version

Version              : 2.13.0
Branch               : 2022.1
Hash                 : ca4bdf101cd1d698377f5a7113d8583f709c2143
Hash Date            : 2025-02-27 06:03:19
XOCL                 : 2.18.179, 3ade2e671e5ab463400813fc2846c57edf82bb10
XCLMGMT              : 2.18.179, 3ade2e671e5ab463400813fc2846c57edf82bb10


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [14]:
from qkeras import QDense, QActivation, quantized_bits, quantized_relu
from tensorflow.keras.models import Sequential
import numpy as np

X = np.random.rand(1000, 10)
y = np.random.randint(2, size=(1000, 1))

model = Sequential()
model.add(
    QDense(
        64,
        input_shape=(10,),
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(QActivation(activation=quantized_relu(6)))
model.add(
    QDense(
        32,
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(QActivation(activation=quantized_relu(6)))
model.add(
    QDense(
        1,
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(Activation('sigmoid'))  # Sigmoid can't be quantized directly (but hls4ml handles it fine)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=5, batch_size=32, verbose=1)

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 0.6948 - accuracy: 0.5160
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.6922 - accuracy: 0.5060
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.6908 - accuracy: 0.5290
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.6891 - accuracy: 0.5320
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.6883 - accuracy: 0.5330


In [15]:
import hls4ml
import pprint

# Generate config
config = hls4ml.utils.config_from_keras_model(model, default_precision='ap_fixed<16,6>')

# Add U55C-specific settings
config['Backend'] = 'Vitis'
config['Part'] = 'xcu55c-fsvh2892-2L-e'
config['ProjectName'] = 'hls4ml_prj_u55c_new'
config['ClockPeriod'] = 5  # You can adjust this if needed
config['IOType'] = 'io_parallel'

pprint.pprint(config)

Interpreting Sequential
Topology:
Layer name: q_dense_3_input, layer type: InputLayer, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 64]
Layer name: q_activation_2, layer type: Activation, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 64]], output shape: [None, 32]
Layer name: q_activation_3, layer type: Activation, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: q_dense_5, layer type: QDense, input shapes: [[None, 32]], output shape: [None, 1]
Layer name: activation_1, layer type: Activation, input shapes: [[None, 1]], output shape: [None, 1]
{'Backend': 'Vitis',
 'ClockPeriod': 5,
 'IOType': 'io_parallel',
 'Model': {'BramFactor': 1000000000,
           'Precision': {'default': 'ap_fixed<16,6>'},
           'ReuseFactor': 1,
           'Strategy': 'Latency',
           'TraceOutput': False},
 'Part'

In [16]:
# Convert the model
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       io_type='io_parallel',
                                                       output_dir='hls4ml_prj_u55c_new',
                                                       part='xcu55c-fsvh2892-2L-e',
                                                       backend='Vitis',)


Interpreting Sequential
Topology:
Layer name: q_dense_3_input, layer type: InputLayer, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 64]
Layer name: q_activation_2, layer type: Activation, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 64]], output shape: [None, 32]
Layer name: q_activation_3, layer type: Activation, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: q_dense_5, layer type: QDense, input shapes: [[None, 32]], output shape: [None, 1]
Layer name: activation_1, layer type: Activation, input shapes: [[None, 1]], output shape: [None, 1]
Creating HLS model


In [ ]:
hls_model.build(csim=False,synth=True,export=True)

In [ ]:
#after generating .xo file, move it inside hls4ml_prj_u55c_new/xo_files

!v++ -c -t hw --platform xilinx_u55c_gen3x16_xdma_3_202210_1 -k myproject -o myproject.xo ./hls4ml_prj_u55c_new/firmware/myproject.cpp

In [ ]:
# modify hls4ml/backends/vitis/vitis_backend.py
# copy 'def make_xclbin()' from hls4ml/backends/vivado_accelerator/vivado_accelerator_backend.py
# or manually link the xclbin file by 'v++ -t -l' command

hls4ml.backends.VitisBackend.make_xclbin(hls_model, hls_model, 'xilinx_u55c_gen3x16_xdma_3_202210_1')